# Impractable transfer learning tflearn example

In [1]:
import numpy             as np
import pandas            as pd
import matplotlib
import os
from   os      import getcwd
from   os      import listdir
from   os.path import isfile, join, isdir

import tensorflow as tf

from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.metrics import Accuracy
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

# `Fish Setup`

In [2]:
trainPath      = '../data/raw/train'
testPath       = '../data/raw/test_stg1'
rawdataPath    = '../data/raw'
fish_classes   = [f for f in listdir(trainPath) if isdir(join(trainPath, f))]
NUM_CATEGORIES  = len(fish_classes)

## `build hdf5 image dataset`

Set BUILD_HDF5_DATASET to True if have not created dataset yet

In [3]:
%%time
#If the training images differ in size to the image input layer of the pretrained network, 
# then you must resize or crop the image data
BUILD_HDF5_DATASET = True
IMAGE_SIZE         = 32
VALIDATION_SPLIT   = True
output_path        = join(rawdataPath, 'fish_dataset_{}x{}.h5'.format(IMAGE_SIZE, IMAGE_SIZE))
input_path         = join(rawdataPath, 'train')

if BUILD_HDF5_DATASET:
    # Build a HDF5 dataset (only required once)
    from tflearn.data_utils import build_hdf5_image_dataset


    build_hdf5_image_dataset(target_path        =input_path, 
                             image_shape        =(IMAGE_SIZE, IMAGE_SIZE), 
                             mode               ='folder', 
                             output_path        =output_path, 
                             categorical_labels =True, 
                             normalize          =True)

CPU times: user 2min 25s, sys: 937 ms, total: 2min 26s
Wall time: 2min 26s


## `load fish X Y`

In [4]:
%%time
from   sklearn.model_selection    import train_test_split

# Load HDF5 dataset
import h5py

h5f         = h5py.File(output_path, 'r')
X_all       = h5f['X'][()]
Y_all       = h5f['Y'][()]

# Split into 
if VALIDATION_SPLIT:
    X, X_valid, Y, Y_valid = train_test_split(X_all, Y_all, 
                                                          test_size    =0.2, 
                                                          random_state =23, 
                                                          stratify     =Y_all)

CPU times: user 131 ms, sys: 87.5 ms, total: 218 ms
Wall time: 253 ms


## `Train a model on the CIFAR-10 dataset and save it` 

In [5]:
# Data loading and organization
from tflearn.datasets import cifar10

(CX, CY), (CX_test, CY_test) = cifar10.load_data()
CX, CY = shuffle(CX, CY)
CY = to_categorical(CY, 10)
CY_test = to_categorical(CY_test, 10)

In [6]:
# train and save a model
with tf.Graph().as_default():


    # Real-time data preprocessing
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    # Real-time data augmentation
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)

    # Convolutional network building
    network = input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    model = tflearn.DNN(network, tensorboard_verbose=0,checkpoint_path='cifar10_cnn.tfl.ckpt')

    # Train using classifier
    model.fit(CX, CY, n_epoch=1, shuffle=True, validation_set=(CX_test, CY_test),
              show_metric=True, batch_size=100, run_id='cifar10_cnn')

    model.save("cifar10_cnn.tfl")

Training Step: 499  | total loss: 1.27351 | time: 19.335s
| Adam | epoch: 001 | loss: 1.27351 - acc: 0.5454 -- iter: 49900/50000
Training Step: 500  | total loss: 1.27581 | time: 20.781s
| Adam | epoch: 001 | loss: 1.27581 - acc: 0.5478 | val_loss: 1.10584 - val_acc: 0.6080 -- iter: 50000/50000
--
INFO:tensorflow:/Users/stokesjd/repos/fish/sashimdig/notebooks/cifar10_cnn.tfl.ckpt-500 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/Users/stokesjd/repos/fish/sashimdig/notebooks/cifar10_cnn.tfl is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
# load that trained model and rerun

with tf.Graph().as_default():


    # Real-time data preprocessing
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    # Real-time data augmentation
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)

    # Convolutional network building
    network = input_data(shape=[None, 32, 32, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    model_ref = tflearn.DNN(network, tensorboard_verbose=0)
    model_ref.load('cifar10_cnn.tfl')
    # Train using classifier
    model_ref.fit(CX, CY, n_epoch=1, shuffle=True, validation_set=(CX_test, CY_test),
              show_metric=True, batch_size=100, run_id='cifar10_cnn_refine')


Training Step: 999  | total loss: 1.02793 | time: 19.073s
| Adam | epoch: 001 | loss: 1.02793 - acc: 0.6336 -- iter: 49900/50000
Training Step: 1000  | total loss: 1.01495 | time: 20.530s
| Adam | epoch: 001 | loss: 1.01495 - acc: 0.6373 | val_loss: 0.93721 - val_acc: 0.6706 -- iter: 50000/50000
--


In [9]:
# now load that model and run on the fish data. choose to not load weights using restore=False

with tf.Graph().as_default():


    # Real-time data preprocessing
    img_prep = ImagePreprocessing()
    img_prep.add_featurewise_zero_center()
    img_prep.add_featurewise_stdnorm()

    # Real-time data augmentation
    img_aug = ImageAugmentation()
    img_aug.add_random_flip_leftright()
    img_aug.add_random_rotation(max_angle=25.)

    # Convolutional network building
    network = input_data(shape=[None, IMAGE_SIZE, IMAGE_SIZE, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
    network = conv_2d(network, 32, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2)
    network = fully_connected(network, 512, activation='relu',restore=False)
    network = dropout(network, 0.5)
    network = fully_connected(network, NUM_CATEGORIES, activation='softmax',restore=False)
    network = regression(network, optimizer='adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.001)
    model_ref = tflearn.DNN(network, tensorboard_verbose=0)
    model_ref.load('cifar10_cnn.tfl')
    # Train using classifier
    model_ref.fit(X, Y, n_epoch=1, shuffle=True, validation_set=(X_valid, Y_valid),
              show_metric=True, batch_size=5, run_id='fish_cnn_refine')



Training Step: 1104  | total loss: 1.19064 | time: 7.913s
| Adam | epoch: 001 | loss: 1.19064 - acc: 0.5469 -- iter: 3020/3021
Training Step: 1105  | total loss: 1.31475 | time: 9.250s
| Adam | epoch: 001 | loss: 1.31475 - acc: 0.5322 | val_loss: 1.17302 - val_acc: 0.5728 -- iter: 3021/3021
--
